<a href="https://colab.research.google.com/github/rineuman/GB_ML_Approach/blob/master/Gro%C3%9FerBeleg2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>